In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import sqlite3
from tabulate import tabulate
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objetivo do projeto
****
Desenvolver um modelo capaz de **prever quais usuários têm maior probabilidade de fazer uma segunda compra em até 30 dias após a realização da primeira**. Segundo um estudo feito no passado, estes usuários possuem maior probabilidade de **fidelizar** na empresa e passar a realizar aquisições mensalmente.

# Conjunto de dados públicos de comércio eletrônico brasileiro por Olist
****
Este é um conjunto de dados público de comércio eletrônico brasileiro de pedidos feitos na Olist Store. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Seus recursos permitem visualizar um pedido de várias dimensões: desde o status do pedido, preço, pagamento e desempenho do frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também lançamos um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas lat/lng.

Estes são dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.

*url: www.kaggle.com/datasets/olistbr/brazilian-ecommerce*

## Sobre a Olist
Olist é uma plataforma de comércio eletrônico brasileira que conecta pequenas e médias empresas a grandes marketplaces, como Mercado Livre, Americanas.com, Submarino, Shoptime, Amazon, entre outros. A plataforma foi criada em 2015 e tem como objetivo ajudar os empreendedores a aumentarem suas vendas online, oferecendo uma solução completa que inclui integração com os principais marketplaces, gestão de estoque, gestão de pedidos, logística e suporte ao cliente.

# Pré-processamento dos dados
****
Nessa etapa, será realizado o pré-processamento dos dados para torná-lo adequados para o uso em um modelo de machine learning, como por exemplo, a criação de features e **target**, divisão entre a base de treino e teste, limpeza de dados. Será utilizado a Interface DB-API para banco de dados SQLite.
## Qual o cenário considerado para criar a base de dados?
Dado que um novo cliente fez uma compra na plataforma e teve sua compra finalizada com a entrega do produto,será coletado:
- Localização do cliente;
- Reputação do(s) produto(s) comprados;    
- Reputação da categoria do(s) produto(s) comprados;
- Reputação do vendedor;
- Informações de entrega;
- Detalhes do valor da venda;
- Detalhes do pagamento;
- **[TARGET]** Flag indicando se o cliente fez uma segunda compra em até 30 dias após a primeira compra

**BASE DE DADOS ESPERADA AO FIM DO PRÉ-PROCESSAMENTO SQL**   

| customer_unique_id               | customer_city         | customer_state | mean_products_review_score | mean_category_products_review_score | mean_seller_review_score | delivery_day_and_except_day | late_delivery | payment_sequential(type) | payment_installments | payment_value | freight_percent | **new_purchase_within_30_days** |
|----------------------------------|-----------------------|----------------|----------------------------|-------------------------------------|--------------------------|-----------------------------|---------------|--------------------------|----------------------|---------------|-----------------|---------------------------------|
| 861eff4711a542e4b93843c6dd7febb0 | franca                | SP             | 3.5                        | 4.7                                 | 2.9                      | -7                          | 0             | credit_card              | 8                    | 99.33         | 0.15            | 1                               |
| 290c77bc529b7ac935b93aa66c333dc3 | sao bernardo do campo | SP             | 2.2                        | 3.4                                 | 3.0                      | 3                           | 1             | credit_card              | 2                    | 107.78        | 0.23            | 0                               |



*explique como você pré-processou os dados para torná-los adequados para o uso em um modelo de machine learning. Por exemplo, você pode ter feito uma limpeza dos dados, normalizado as colunas, separado em conjuntos de treino e teste, entre outras coisas.*


## Criando e populando o banco de dados

In [2]:
conn   = sqlite3.connect('OListDB')
cursor = conn.cursor()

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        tablename = filename.replace('.csv', '').replace('olist_', '').replace('_dataset', '').upper()
        df = pd.read_csv(os.path.join(dirname, filename))
        df.to_sql(tablename, conn, if_exists='replace', index=False)  
        print(f'[OK] {filename} TO {tablename}')

[OK] olist_customers_dataset.csv TO CUSTOMERS
[OK] olist_sellers_dataset.csv TO SELLERS
[OK] olist_order_reviews_dataset.csv TO ORDER_REVIEWS
[OK] olist_order_items_dataset.csv TO ORDER_ITEMS
[OK] olist_products_dataset.csv TO PRODUCTS
[OK] olist_geolocation_dataset.csv TO GEOLOCATION
[OK] product_category_name_translation.csv TO PRODUCT_CATEGORY_NAME_TRANSLATION
[OK] olist_orders_dataset.csv TO ORDERS
[OK] olist_order_payments_dataset.csv TO ORDER_PAYMENTS


In [42]:
def get_query(SQL: str, tablefmt: str = 'github') -> str:
    query = cursor.execute(SQL)
    
    cols = [col[0] for col in query.description]
    rows = query.fetchall()
    
    return tabulate(rows, cols, tablefmt = tablefmt)

In [43]:
SQL = \
'''
SELECT * FROM CUSTOMERS LIMIT 5
'''

query = get_query(SQL)
print(query)

| customer_id                      | customer_unique_id               |   customer_zip_code_prefix | customer_city         | customer_state   |
|----------------------------------|----------------------------------|----------------------------|-----------------------|------------------|
| 06b8999e2fba1a1fbc88172c00ba8bc7 | 861eff4711a542e4b93843c6dd7febb0 |                      14409 | franca                | SP               |
| 18955e83d337fd6b2def6b18a428ac77 | 290c77bc529b7ac935b93aa66c333dc3 |                       9790 | sao bernardo do campo | SP               |
| 4e7b3e00288586ebd08712fdd0374a03 | 060e732b5b29e8181a18229c7b0b2b5e |                       1151 | sao paulo             | SP               |
| b2b6027bc5c5109e529d4dc6358b12c3 | 259dac757896d24d7702b9acbbff3f3c |                       8775 | mogi das cruzes       | SP               |
| 4f2d8ab171c80ec8364f7c12e35b23ad | 345ecd01c38d18a9036ed96c73b8d066 |                      13056 | campinas              | SP         

In [44]:
SQL = \
'''
SELECT * FROM SELLERS LIMIT 5
'''

query = get_query(SQL)
print(query)

| seller_id                        |   seller_zip_code_prefix | seller_city       | seller_state   |
|----------------------------------|--------------------------|-------------------|----------------|
| 3442f8959a84dea7ee197c632cb2df15 |                    13023 | campinas          | SP             |
| d1b65fc7debc3361ea86b5f14c68d2e2 |                    13844 | mogi guacu        | SP             |
| ce3ad9de960102d0677a81f5d0bb7b2d |                    20031 | rio de janeiro    | RJ             |
| c0f3eea2e14555b6faeea3dd58c1b1c3 |                     4195 | sao paulo         | SP             |
| 51a04a8a6bdcb23deccc82b0b80742cf |                    12914 | braganca paulista | SP             |


In [45]:
SQL = \
'''
SELECT * FROM ORDER_REVIEWS LIMIT 5
'''

query = get_query(SQL)
print(query)

| review_id                        | order_id                         |   review_score | review_comment_title   | review_comment_message                                                                               | review_creation_date   | review_answer_timestamp   |
|----------------------------------|----------------------------------|----------------|------------------------|------------------------------------------------------------------------------------------------------|------------------------|---------------------------|
| 7bc2406110b926393aa56f80a40eba40 | 73fc7af87114b39712e6da79b0a377eb |              4 |                        |                                                                                                      | 2018-01-18 00:00:00    | 2018-01-18 21:46:59       |
| 80e641a11e56f04c1ad469d5645fdfde | a548910a1c6147796b98fdf73dbeba33 |              5 |                        |                                                                             

In [46]:
SQL = \
'''
SELECT * FROM ORDER_ITEMS LIMIT 5
'''

query = get_query(SQL)
print(query)

| order_id                         |   order_item_id | product_id                       | seller_id                        | shipping_limit_date   |   price |   freight_value |
|----------------------------------|-----------------|----------------------------------|----------------------------------|-----------------------|---------|-----------------|
| 00010242fe8c5a6d1ba2dd792cb16214 |               1 | 4244733e06e7ecb4970a6e2683c13e61 | 48436dade18ac8b2bce089ec2a041202 | 2017-09-19 09:45:35   |   58.9  |           13.29 |
| 00018f77f2f0320c557190d7a144bdd3 |               1 | e5f2d52b802189ee658865ca93d83a8f | dd7ddc04e1b6c2c614352b383efe2d36 | 2017-05-03 11:05:13   |  239.9  |           19.93 |
| 000229ec398224ef6ca0657da4fc703e |               1 | c777355d18b72b67abbeef9df44fd0fd | 5b51032eddd242adc84c38acab88f23d | 2018-01-18 14:48:30   |  199    |           17.87 |
| 00024acbcdf0a6daa1e931b038114c75 |               1 | 7634da152a4610f1595efa32f14722fc | 9d7a1d34a5052409006425275

In [47]:
SQL = \
'''
SELECT * FROM PRODUCTS LIMIT 5
'''

query = get_query(SQL)
print(query)

| product_id                       | product_category_name   |   product_name_lenght |   product_description_lenght |   product_photos_qty |   product_weight_g |   product_length_cm |   product_height_cm |   product_width_cm |
|----------------------------------|-------------------------|-----------------------|------------------------------|----------------------|--------------------|---------------------|---------------------|--------------------|
| 1e9e8ef04dbcff4541ed26657ea517e5 | perfumaria              |                    40 |                          287 |                    1 |                225 |                  16 |                  10 |                 14 |
| 3aa071139cb16b67ca9e5dea641aaa2f | artes                   |                    44 |                          276 |                    1 |               1000 |                  30 |                  18 |                 20 |
| 96bd76ec8810374ed1b65e291975717f | esporte_lazer           |                    46 |      

In [48]:
SQL = \
'''
SELECT * FROM GEOLOCATION LIMIT 5
'''

query = get_query(SQL)
print(query)

|   geolocation_zip_code_prefix |   geolocation_lat |   geolocation_lng | geolocation_city   | geolocation_state   |
|-------------------------------|-------------------|-------------------|--------------------|---------------------|
|                          1037 |          -23.5456 |          -46.6393 | sao paulo          | SP                  |
|                          1046 |          -23.5461 |          -46.6448 | sao paulo          | SP                  |
|                          1046 |          -23.5461 |          -46.643  | sao paulo          | SP                  |
|                          1041 |          -23.5444 |          -46.6395 | sao paulo          | SP                  |
|                          1035 |          -23.5416 |          -46.6416 | sao paulo          | SP                  |


In [49]:
SQL = \
'''
SELECT * FROM PRODUCT_CATEGORY_NAME_TRANSLATION LIMIT 5
'''

query = get_query(SQL)
print(query)

| product_category_name   | product_category_name_english   |
|-------------------------|---------------------------------|
| beleza_saude            | health_beauty                   |
| informatica_acessorios  | computers_accessories           |
| automotivo              | auto                            |
| cama_mesa_banho         | bed_bath_table                  |
| moveis_decoracao        | furniture_decor                 |


In [50]:
SQL = \
'''
SELECT * FROM ORDERS LIMIT 5
'''

query = get_query(SQL)
print(query)

| order_id                         | customer_id                      | order_status   | order_purchase_timestamp   | order_approved_at   | order_delivered_carrier_date   | order_delivered_customer_date   | order_estimated_delivery_date   |
|----------------------------------|----------------------------------|----------------|----------------------------|---------------------|--------------------------------|---------------------------------|---------------------------------|
| e481f51cbdc54678b7cc49136f2d6af7 | 9ef432eb6251297304e76186b10a928d | delivered      | 2017-10-02 10:56:33        | 2017-10-02 11:07:15 | 2017-10-04 19:55:00            | 2017-10-10 21:25:13             | 2017-10-18 00:00:00             |
| 53cdb2fc8bc7dce0b6741e2150273451 | b0830fb4747a6c6d20dea0b8c802d7ef | delivered      | 2018-07-24 20:41:37        | 2018-07-26 03:24:27 | 2018-07-26 14:31:00            | 2018-08-07 15:27:45             | 2018-08-13 00:00:00             |
| 47770eb9100c2d0c44946d9cf07ec65d |

In [51]:
SQL = \
'''
SELECT * FROM ORDER_PAYMENTS LIMIT 5
'''

query = get_query(SQL)
print(query)

| order_id                         |   payment_sequential | payment_type   |   payment_installments |   payment_value |
|----------------------------------|----------------------|----------------|------------------------|-----------------|
| b81ef226f3fe1789b1e8b2acac839d17 |                    1 | credit_card    |                      8 |           99.33 |
| a9810da82917af2d9aefd1278f1dcfa0 |                    1 | credit_card    |                      1 |           24.39 |
| 25e8ea4e93396b6fa0d3dd708e76c1bd |                    1 | credit_card    |                      1 |           65.71 |
| ba78997921bbcdc1373bb41e913ab953 |                    1 | credit_card    |                      8 |          107.78 |
| 42fdf880ba16b47b59251dd489d4441a |                    1 | credit_card    |                      2 |          128.45 |


# Análise exploratória dos dados
****
*forneça uma breve análise exploratória dos dados. Isso pode incluir a distribuição das classes de destino, uma matriz de correlação entre as variáveis, visualizações de distribuições, entre outros*

# Modelagem
****
*explique o modelo que você escolheu e por que o escolheu. Explique também os hiperparâmetros que você usou.*

# Resultados
****
*apresente os resultados do seu modelo, como a acurácia ou a precisão. Você pode incluir também outras métricas que sejam relevantes para o seu problema.*

# Conclusão
****
*faça uma conclusão sobre o seu projeto, destacando seus resultados e eventuais limitações ou desafios que você enfrentou durante o processo.*  
Neste projeto tive a limitação de não trabalhar com o PL/SQL. O que dificultou no desenvolvimento das métricas na etapa de pré-processamento dos dados. No uso desse modelo na vida real, a escrita de código procedural no banco de dados com o PL/SQL seria essencial para executar tarefas mais complexas, como processamento de dados, lógica de negócios, manipulação de exceções, entre outras coisas.